# 3. Flask - Generating HTML Pages & Template Creation (Jinja2)
We will move from basic reports (static HTML) to a full-fledged graphical interface for our control system. We will learn how to use the Jinja2 templating language to insert `dynamic data` and logic into HTML, and create reusable templates, allowing us to generate complex, full-featured HTML pages instead of returning simple text strings. We will also start loading the configuration for our Mission Control from files.

- Configuration from a file
- Jinja2 - templating language
- `Dynamic generation` of HTML pages
- Inserting logic and control structures
- Template extension and inheritance
- Route organization

`Application structure`:
- my_project/
  - my_app.py
  - **routes.py**
  - templates/
  - static/
  - **config/**

## 3.1. Configuration from a File
Every mission requires settings that can be easily changed without interfering with the main systems. We will move our settings from the main program into a separate file.

- Organized and clear
- Configuration and settings are in `one place`
- Suitable for `real projects` and `applications`
- P.S. When using a class, we can utilize inheritance to have different configuration settings.

#### Configuration files in a folder, e.g., **config/**

In [ ]:
# app_config_constants.py

# General Config
DEBUG = True # setting for debugging

# the "templates" and "static" parameters are set on the app object, not here!
# we can leave them as default, i.e., template_folder = "templates", static_folder = "static"

In [ ]:
# app_config_class.py

class Config:
    """
    Set Flask config variables
    """
    # General Config
    DEBUG = True # setting for debugging

    # the "templates" and "static" parameters are set on the app object, not here!
    # we can leave them as default, i.e., template_folder = "templates", static_folder = "static"

In [ ]:
# my_app.py
# using constants


from flask import Flask


app = Flask(__name__) # here we can specify custom (or leave default) folders for template_folder and static_folder

# -------------- CONFIGURATION --------------
# from a file using "constants"
app.config.from_pyfile("config/app_config_constants.py")
print(app.config) # prints the application's configuration


# -------------- HOMEPAGE --------------

@app.route("/")
def index():
    return f"""
    I know how to configure from constants !
    """

# -------------- STARTING THE APP --------------
if __name__ == "__main__":
    app.run() # we don't need to specify debug=True

In [ ]:
# my_app.py
# using a class


from flask import Flask
from config.app_config_class import Config # we import the class


app = Flask(__name__) # here we can specify custom (or leave default) folders for template_folder and static_folder

# -------------- CONFIGURATION --------------
# from a file using a "class"
app.config.from_object(Config)
print(app.config) # prints the application's configuration


# -------------- HOMEPAGE --------------

@app.route("/")
def index():
    return f"""
    I know how to configure from class !
    """

# -------------- STARTING THE APP --------------
if __name__ == "__main__":
    app.run() # we don't need to specify debug=True

## 3.2. Jinja2 Template Engine
We need to be able to display data in real-time without changing the entire design each time. We will get acquainted with the `Jinja2` templating engine, which will allow us to "inject" **dynamic data** from our Python scripts directly into prepared `HTML` schemas.

### 3.2.1. JINJA2 Control Structures & Logic
- Allows inserting basic control structures (conditions, loops) and generated results into an HTML page.

### JINJA2
1.  **`VARIABLES, FUNCTIONS, VALUES`**
    - `{{ ... }}` for Expressions = `{{ parameter/number/result/function() }}`

2.  **`CONTROL STRUCTURES`**
    - CONDITIONS
      - `{% if ... %}`
      - `{% elif ... %}`
      - `{% else ... %}`
      - `{% endif %}`

    - FOR loops
      - `{% for .. in .. %}` - start of a multi-line block
        - `{{ parameter / function result }}` - statement for a single-line output
      - `{% endfor %}` - end of a multi-line block

In [ ]:
# my_app.py

from flask import Flask, render_template, url_for
import time

app = Flask(__name__)


# -------------- CONFIGURATION --------------
# from a file using constants
app.config.from_pyfile("config/app_config_constants.py")


# -------------- DYNAMIC CONTENT - JINJA TEMPLATE --------------
@app.route("/probe") 
def basic_info():  # I'll create a variable to pass to the HTML page
    satellite = {
        "name": "Low Earth Orbit Satellite",
        "factory": "Moon LEOSat",
        "status": "Launched",
        "date": time.asctime(),
        "equipment": ["IFR camera", "Gravity sensor", "Plasma propeller", "Solar panels", "Radio", "Thrust vectoring"]
    }
    
    return render_template("probe.html", probe=satellite) 
    # returns the content of "probe.html" and passes the "probe" variable with the value of "satellite" (a dictionary) into it

# url_for('function_name') - for functions without parameters
# url_for('function_name', url_parameter_name=variable_value) - with parameters
# dynamic creation of URL addresses/links on our website - e.g., for navigation
@app.route("/url_practise") 
def show_url():  
    url = url_for("basic_info") # creates a URL link from the route of the "basic_info" FUNCTION
    return render_template("myURL.html", given_url=url) 
    # The URL link is passed to the page as a KEYWORD argument "given_url"

# -------------- STARTING THE APP --------------
if __name__ == "__main__":
    app.run()

#### HTML pages are in the **templates/** folder

In [ ]:
<!-- probe.html -->

<!DOCTYPE html>
<html>

<head>
    <meta charset="utf-8" />
</head>

<body>
    <h1>Probe {{ probe["name"] }}</h1>

    <p><h2>Manufactured by {{ probe["factory"] }}</h2></p>
    
    <p>Detailed information:</p>
    {% for item in probe["equipment"] %}
        <p>Installed equipment: {{ item }}</p>
    {% endfor %}

    <br>

    <h3>Launch status:</h3>
    {% if probe["status"] == "Launched" %}
        <p>The {{ probe["name"] }} is already launched !</p>
        <p>Probe was launched on: {{ probe["date"] }}</p>
    {% elif probe["status"] == "Canceled" %}
        <p>The {{ probe["name"] }} was canceled</p>
    {% else %}
        <p>The {{ probe["name"] }} is being prepared for execution</p>
    {% endif %}
    
</body>

</html>

In [ ]:
<!-- myURL.html -->
 
<!DOCTYPE html>
<html>

<head>
    <meta charset="utf-8" />
</head>

<body>
    <p>This is my Probe page: "{{ given_url }}"</p> <!-- get URL from a passed function -->

    <p><a href="{{ url_for('basic_info') }}">Go to probe details with url_for</a></p>
    <!-- use URL from a passed function to create a link -->

    <!-- if the function accepts parameters we have to pass them like this:
     {{ url_for('function_name', parameter_name_in_url=value) }} 
    -->

</body>

</html>

### **Practise I**

**Sensor Status Report**

We will create an application and a `/sensor_status` route in it. This route will pass a list of several sensors to the `sensors.html` HTML template. Each sensor will be a dictionary with `name` and `status` keys.

- In the `sensors.html` template, we will then use JINJA to loop through all the sensors and print their names.
- If a sensor has a `status` set to `"offline"`, we will display a **"WARNING: OFFLINE"** warning next to its name.

**Data:**
```python
sensor_data = [
        {'name': 'Core temperature', 'status': 'online'},
        {'name': 'Cabin pressure', 'status': 'online'},
        {'name': 'Gyroscope', 'status': 'offline'},
        {'name': 'Solar panels', 'status': 'online'}
    ]
```

### 3.2.2 Template Extension and Inheritance
- I will create `smaller elements` (navigation, footer) - e.g., "header.html", "footer.html"
- I will create a `base (parent) template` - typically "base.html" or "layout.html"
  - Into the "base.html" template, I will insert these smaller elements - using **{% include 'navbar.html' %}**
  - I will also define blocks for descendants - using **{% block content %}" and "{% endblock %}"**
- In the inheriting templates, I will insert/overwrite the content of these blocks

- In the BASE template, we define **common** elements and a unified look; in the descendants, we insert **individual** content.
- If we decide, for example, to change the look, we only need to change the BASE template, and because the descendants inherit from it, the change will automatically apply to them = `I don't have to change each page separately`.

### **JINJA 2**
- `EXTENSIONS`
  - `{% include "navigation.html" %}`
  - `{% extends "document.html" %}`

- `BLOCKS`
  - `{% block article_content %}`
    - .. text and content of the article ..
  - `{% endblock article_content %}`

- `COMMENTS`
- `{# ... #}` comment, will not be displayed

In [ ]:
# my_app.py

from flask import Flask, render_template, url_for
import datetime


app = Flask(__name__)


# -------------- CONFIGURATION --------------
# from a file - using constants
app.config.from_pyfile("config/app_config_constants.py")


# -------------- DYNAMIC CONTENT - JINJA TEMPLATE --------------
@app.route("/mission_details") 
def mission():  # I'll create a variable to pass to the HTML page
    launch_date = datetime.datetime.now()+datetime.timedelta(days=20)
    mission = {
        "mission_name": "Dimorphos 2035",
        "launch_vehicle": "Skylla IV",
        "destination": "Dimorphos",
        "launch_date": datetime.datetime.strftime(launch_date, "%d. %m. %Y %H:%M:%S"),
    }
    return render_template("mission.html", mission=mission)
    # returns the content of "mission.html" and passes the "mission" variable with the value of "mission" (a dictionary) into it
    

# -------------- STARTING THE APP --------------
if __name__ == "__main__":
    app.run()

#### HTML pages are in the **templates/** folder

In [ ]:
<!-- header.html -->
 
<nav>
    <ul> <!-- all routes (and functions) have to exist for every page we want to create a link for -->
        <li><a href="{{ url_for('mission_home') }}">Mission Home</a></li> 
        <li><a href="{{ url_for('about') }}">About Team</a></li>
        <li><a href="{{ url_for('contact') }}">Contact Us</a></li>
    </ul>
</nav>

In [ ]:
<!-- footer.html -->
 
<footer>
    <br>
    <p>Asteroid mining inc.</p>
    <p>All rights reserved</p>
</footer>

In [ ]:
<!-- base.html -->
 
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>{% block page_title %} Title {% endblock page_title %}</title> <!-- dynamic content -->
    
</head>

<body>
    {% include 'header.html' %} <!-- we include the content of "header.html"  -->
    {% block page_content %} Content {% endblock page_content %} <!-- dynamic content -->
    {% include 'footer.html' %} <!-- we include the content of "footer.html" -->
</body>

</html>

In [ ]:
<!-- mission.html -->
 
{% extends "base.html" %} <!-- we inherit content of "base.html" -->

<!-- add dynamic content into predefined blocks -->
{% block page_title %} {{ mission["mission_name"] }} {% endblock page_title %} <!-- add dynamic content - block Title -->

{% block page_content %} <!-- add dynamic content - block page_content -->
    <p>Brief description</p>
    <p>Launch site: {{ mission["destination"] }}</p> <!-- values from the "mission" variable -->
    <p>Vehicle: {{ mission["launch_vehicle"] }}</p>
    <p>Mission details:</p>
    <p>Launch date: {{ mission["launch_date"] }}</p>
{% endblock page_content %}

## 3.3. Organizing Routes
When the number of navigation routes becomes unmanageable, we will move them to a separate "protocol" file, from which our program will simply import them.

- In the case of a small number of routes, we can leave them in the same file as the application.

In [ ]:
# routes.py

def register_routes(app): # function with a parameter (for the future application instance)
    @app.route("/earth") 
    def mission_earth(): # all routes are placed inside the "wrapper" function
        return """
            <h1>Mission</h1>
            <p>Welcome to Earth !</p>
        """
    
    @app.route("/mars")
    def mission_mars(): # all routes are placed inside the "wrapper" function
        return """
            <h1>Mission</h1>
            <p>Welcome to Mars !</p>
        """
    
    @app.route("/venus")
    def mission_venus(): # all routes are placed inside the "wrapper" function
        return """
            <h1>Mission</h1>
            <p>Welcome to Venus !</p>
        """

In [ ]:
# my_app.py

from flask import Flask
from routes import register_routes


app = Flask(__name__)


# -------------- CONFIGURATION --------------
# from a file - using constants
app.config.from_pyfile("config/app_config_constants.py")


# -------------- INITIALIZATION --------------
register_routes(app) # connecting the "register_routes" function with the application (which we pass as a parameter)
    

# -------------- STARTING THE APP --------------
if __name__ == "__main__":
    app.run()

### **Practise II**
**Standardized Report Format**

We will create a standardized format for mission reports using template inheritance.

1.  **Create `base_report.html`:** This will be the base (parent) template. It must contain:
    * A heading `<h1>Mission Report</h1>`
    * A block for content
    * A footer `<p>-- End of Transmission --</p>`

2.  **Create `daily_log.html`:** This will be the inheriting (child) template.
    * At the beginning, it must "inherit" everything from `base_report.html`.
    * Then, it must overwrite the content of the block with its own text, for example: `<h2>Daily Log: Day 5</h2><p>Solar wind levels are stable.</p>`.

3.  **In `my_app.py`**, create a simple route `/daily_log` that only displays the `daily_log.html` template.

### **Project (Homework): Central Data Hub Implementation**

**Mission:** Enhance the information panel from the previous mission with a unified structure and dynamic display of crew data. The goal is to apply knowledge of the Jinja2 templating system and increase the application's modularity.

`Basic application structure`:
- my_project/
  - my_app.py
  - **routes.py**
  - templates/
  - static/
  - **config/**


1. **Creating a Standardized Interface (Master Template):**
  - Create a base template `base.html` that will define a unified look for the entire information panel.
  - Create partial templates `header.html` (for the navigation bar with links) and `footer.html` (for your corporation's copyright footer).
  - Using `{% include ... %}`, insert `header.html` and `footer.html` into the base template `base.html`.
  - In `base.html`, define at least one block for the main page content, e.g., `{% block content %}`.

2.  **Upgrading Existing Panels:**
  - Modify all your templates for the routes (`/`, `/mission_briefing`, `/target_asteroid`) so that they inherit from `base.html`.
  - Insert the unique content of the templates into the `content` block.

3.  **Upgrading Crew Info:**
  - From the existing `/crew_members/<int:id>` route, extract the list of dictionaries with crew data and make it a global variable.
  - Further modify the `/crew_members/<int:id>` route to generate an HTML page using `render_template()`.
  - Then, create a new route and template for `/crew_members`.
  - In the template for the `/crew_members` route, use a loop to display the names of all crew members - each name will also be a **link** - use `url_for()`.
  - This link (for each crew member) will lead to their detailed profile at `/crew_members/<int:id>`.

4.  **Upgrading Asteroid Information:**
  - Remove the route and HTML file for `/target_asteroid/<string:asteroid_name>`.
  - For the `/target_asteroid/` route, move the data from the HTML file to a global variable in our program and pass it to the HTML.

5.  **Optimizing Communication Protocols (Bonus I):**
  - Move all your route definitions to a separate file (e.g., `routes.py`) and import them into your main program.

6.  **Unified Configuration (Bonus II):**
  - Move the configuration from the application to a file in the `config/` folder.

---
#### © Jiří Svoboda (George Freedom)
- Web: https://GeorgeFreedom.com
- LinkedIn: https://www.linkedin.com/in/georgefreedom/
- Book me: https://cal.com/georgefreedom